In [49]:
import warnings
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, fisher_exact, pearsonr, spearmanr, shapiro



warnings.filterwarnings('ignore')

In [50]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [51]:
df = pd.read_csv("churn_analysis.csv") 
df.head()

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. 

Для этого мы воспользуемся критерием хи-квадрат.  Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn.  Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции 

In [52]:
df_control = df[df['treatment'] == 1]
df_control_states = df_control['state'].unique()
pairs = set()
for key1 in df_control_states:
    for key2 in df_control_states:
        if key1 != key2 and (key1, key2) not in pairs and (key2, key1) not in pairs:
                pairs.add((key1, key2))

3       False.
152     False.
156      True.
231     False.
384     False.
454      True.
462     False.
939     False.
1014    False.
1060    False.
1238    False.
1791    False.
1821    False.
1822    False.
1890    False.
2048    False.
2131    False.
2224    False.
2372    False.
2511    False.
2656    False.
2753    False.
2904    False.
3325    False.
Name: churn, dtype: object

In [53]:
obs = {}
for key1, key2 in pairs:
    val1_t = len(df_control[df_control['state'] == key1][df_control['churn'] == 'True.'])
    val1_f = len(df_control[df_control['state'] == key1][df_control['churn'] == 'False.']) 
    val2_t = len(df_control[df_control['state'] == key2][df_control['churn'] == 'True.'])
    val2_f = len(df_control[df_control['state'] == key2][df_control['churn'] == 'False.']) 
    obs[(key1, key2)] = [[val1_t,val2_t], [val1_f, val2_f]]
    


In [54]:
answer1 = 0
obs_answer1 = {}
for key in obs.keys():
    stat, p, *_ = chi2_contingency(obs[key], correction=False)
    if p < 0.05:
        answer1 += 1
    obs_answer1[key] = stat, p

answer1


34


Эта поправка обычно используется для таблиц сопряженности размером 2x2  и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода). 

Полезно знать, что эта поправка часто включена по  умолчанию (например, в функции scipy.stats.chi2_contingency) и понимать ее влияние на оценку достигаемого уровня значимости. 

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией 

In [55]:
answer2 = 0
obs_answer2 = {}
for key in obs.keys():
    stat, p, *_ = chi2_contingency(obs[key], correction=True)
    if p < 0.05:
        answer2 += 1
    obs_answer2[key] = stat, p

answer2

0

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

In [56]:
answer4 = 0
obs_answer4 = {}
for key in obs.keys():
    stat, p, *_ =fisher_exact(obs[key])
    if p < 0.05:
        answer4 += 1
    obs_answer4[key] = stat, p

answer4

10

Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике. 

Рассмотрим пару признаков day_calls и  mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость. 

In [57]:
corr, pval = pearsonr( df['day_calls'],df['mes_estim'])
corr, pval

(-0.05179435058757264, 0.0027798836869738384)

Еще раз рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость

In [58]:
corr, pval = spearmanr( df['day_calls'],df['mes_estim'])
corr, pval

(0.043349880533927444, 0.012317367189170541)

Посчитайте значение коэффицента корреляции Крамера между двумя признаками: штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

In [59]:
chi_stat = []
temp_table = pd.DataFrame(columns = ['state', 'churn_true', 'churn_false'])

for i, state in enumerate(df_control_states):
    temp_table.loc[i] = [state,
                         len(df[df['state'] == state][df['churn'] == 'True.']),
                         len(df[df['state'] == state][df['churn'] == 'False.'])]

temp_table = temp_table.set_index('state')
stat, p_value, _, _ = chi2_contingency(temp_table)

stat, p_value

(83.04379191019666, 0.0022962215520111536)

In [60]:
len(temp_table)

51

In [61]:
v_kramer = (stat**2 / sum(temp_table.sum()) / (len(temp_table) - 1))**0.5
v_kramer

0.20342508799454118